<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Обзор

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

**Стратегия это - функция для открытия и закрытия позиций созданная из индикатора**.  Например, если у вас есть индикатор, отображающий пересечение скользящих средних, вы можете превратить его в стратегию, добавив команды strategy.entry для входа и strategy.close для выхода из позиций. 
т.е. по сути это индикатор или комплекс индикаторов заключенной в одной функции.

<p style = "background-color : #afeeee; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Импортируем библиотеки используемые в работе:</span>

In [1]:
# Расчёты
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Для распечатки в цвете:
from termcolor import colored

from sklearn.preprocessing import MinMaxScaler

import re
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
from collections import Counter

<p style = "background-color : #afeeee; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Функции:</span>

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 5px"><strong>
<span style="color:#000">Графики</span>

In [2]:
def build_graph(library, chart_type, data_frame, x, y, height, width, title, xaxis_title, yaxis_title, legend = None, color = None):
    """Функция по построению графиков

    Args:
        library: Библиотека построения
        chart_type: Тип графика
        data_frame: Датафрейм из которого будет читаться данные
        x: Ось x
        y: Ось y
        height: Высота графика
        width: Ширина графика
        title: Название графика
        xaxis_title: Название оси x
        yaxis_title: Название оси y
        legend: Название легенды
        color: Цвет колонок
    Raises:
        ValueError: Ошибка вызова графика
    """
    if library == 'plt':
        plt.figure(figsize=(height, width))
        plt.title(title)
        if chart_type == 'box':
            sns.boxplot(data=data_frame, x=x, y=y)
        elif chart_type == 'bar':
            sns.barplot(data=data_frame, x=x, y=y)
        else:
            raise ValueError("Неподдерживаемый тип графика. Допустимые значения chart_type: 'box', 'bar'")
        plt.xlabel(xaxis_title)
        plt.ylabel(yaxis_title)
        plt.show()
    # plotly
    elif library == 'px':
        if chart_type == 'box':
            fig = px.box(data_frame=data_frame, x=x, y=y, color=color, title=title)
        elif chart_type == 'bar':
            fig = px.bar(data_frame=data_frame, x=x, y=y, color=color, title=title)
        else:
            raise ValueError("Неподдерживаемый тип графика. Допустимые значения chart_type: 'box', 'bar'")
        
        fig.update_layout(
            title=dict(x=0.5, xanchor="center"),
            legend_title=legend,
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            margin=dict(l=0, r=0, t=30, b=0),
            title_x=0.5
        )
        fig.show()
    else:
        raise ValueError("Неподдерживаемая библиотека. Допустимые значения library: 'px', 'plt'")


In [3]:
def pie_graph(data, title_text):
    # Построение круговой диаграммы
    fig = go.Figure(
        data=[go.Pie(       
            values=data.values, 
            labels=data.index, 
            pull=[0.1]*len(data)
        )]
    )


    fig.update_traces(
        hoverinfo='label+value',  # Добавляем информацию при наведении
        textinfo='percent',
        textfont_size=15,
        marker=dict(line=dict(color='#000000', width=2))
    )

    # Устанавливаем название графика и размеры текста:
    fig.update_layout(
        title_text=title_text,
        title_font_size=20,
        legend_font_size=20,
        width=800,
        height=600
    )

    fig.show()

In [4]:
def gist_graph(df, column, title):    
    # Вычисляем моду, это понадобится для выделение моды на графике
    sharpe_mode = df[column].mode()[0]

    fig = px.histogram(
        df,   
        x = column,
        marginal='box',
        nbins=200,
        width=1000,
        height=500,
        color_discrete_sequence=['Orangered']
    )

    # Устанавливаем название графика, размеры текста и наименование осей:
    fig.update_layout(
        title={
            'text': title,
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
            },
        title_font={
            'size':18,
            'color':"black"
            },
        xaxis_title='Размер коэффициента',
        yaxis_title='Количество вхождений'
    )

    # Добавляем обводку колонок
    fig.update_traces(
        hoverinfo="all",
        hovertemplate="Диапазон размера коэффициента: %{x}",
        marker_line_color='black',   # Цвет линии колонки
        marker_line_width=1.0,       # Толщина линии колонки
        opacity=0.9            # Прозрачность колонки
    )

    # Устанавливаем дополнительную линию моды на графике:
    fig.add_vline(
        x=sharpe_mode,
        line_width=2, 
        line_color="black", 
        annotation_text=f"Мода распределения: {sharpe_mode}",
        annotation_position='bottom left'
    )

    fig.update_xaxes(showspikes=True)
    fig.update_yaxes(showspikes=True)
    fig.show()

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 5px"><strong>
<span style="color:#000">Работа с текстом</span>

In [5]:
# Функция для очистки текста
def clean_text(text):
    # Удаление эмодзи и спецсимволов, кроме пробелов
    text = re.sub(r'[^\w\s]', '', str(text))
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

In [6]:
def classify_tags(row, tags_dict):
    """Функция для классификации тегов на основе нового словаря."""
    text_to_analyze = f"{str(row['strategy_description'])} {str(row['tags'])}"
    classifications = []

    for category, keywords in tags_dict.items():
        if any(re.search(r'\b' + re.escape(keyword) + r'\b', text_to_analyze) for keyword in keywords):
            classifications.append(category)

    # Если найдено более одного тега, возвращаем 'mixed' УБРАТЬ ЭТУ СТРОКУ ЕСЛИ НАМ НЕОБХОДИМЫ ВСЕ ВХОЖДЕНИЯ
    if len(classifications) > 1:
        return 'combined strategy'
    
    return classifications[0] if classifications else 'No classification'

In [7]:
def check_keywords(df, tags):
    """Функция для проверки наличия ключевых слов в датафрейме."""
    # Объединяем описание и теги
    df['combined'] = (df['strategy_description'].fillna('') + ' ' + df['tags'].fillna('')).str.lower()
    
    # Функция для обработки каждой строки
    def process_text(text):
        matched_tags = []
        for keyword in tags:
            if keyword in text:
                matched_tags.append(keyword)
        return matched_tags

    # Применяем функцию к объединенному тексту
    df['type_tags'] = df['combined'].apply(process_text)
    return df.drop(columns=['combined'])

In [8]:
def convert_to_numeric(val):
    # Проверяем, является ли значение строкой
    if isinstance(val, str):
        # Заменяем символы и удаляем пробелы
        val = val.replace('−', '-')  # Заменяем минус на стандартный
        val = val.replace('+', '')    # Удаляем знак плюс
        val = val.replace(',', '')     # Удаляем запятые
        # Проверяем, содержит ли значение знак процента
        if '%' in val:
            val = val.replace('%', '')  # Удаляем знак процента
            val = val.strip()            # Удаляем пробелы по краям
            # Преобразуем строку в числовой формат и делим на 100 для получения десятичного значения
            return pd.to_numeric(val, errors='coerce') / 100
    # Если значение уже числовое, просто возвращаем его
    return val if isinstance(val, (int, float)) else pd.to_numeric(val, errors='coerce')


In [9]:
# def convert_to_numeric(val):
#     # Заменяем символы и удаляем пробелы
#     val = val.replace('−', '-')  # Заменяем минус на стандартный
#     val = val.replace('+', '')    # Удаляем знак плюс
#     val = val.replace(',', '')     # Удаляем запятые
#     val = val.replace('%', '')     # Удаляем проценты
#     return pd.to_numeric(val, errors='coerce')  # для преобразования строки в числовой формат, при этом errors='coerce' заменяет некорректные значения на NaN.

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 5px"><strong>
<span style="color:#000">Создание датафреймов</span>

In [10]:
def process_tags(df, tags_dict):
    """Функция по созданию датафреймов согласно списку тегов."""
    tag_counts_results = {}

    for tag_name, keywords in tags_dict.items():
        # Проверяем ключевые слова из ранее сделанной функции
        filtered_df = check_keywords(df, keywords)
        # Убираем строки с пустыми классификациями
        filtered_df = filtered_df[filtered_df['type_tags'].apply(lambda x: x != [])]
        # Сохраняем только нужные столбцы
        filtered_df = filtered_df[['title', 'type_tags']]
        
        # Разбиваем списки на отдельные строки
        exploded_tags = filtered_df.explode('type_tags')
        
        # Подсчитываем количество вхождений каждого тега
        tag_counts = exploded_tags['type_tags'].value_counts().reset_index()
        tag_counts.columns = ['tag', 'count']  # Переименовываем колонки
        
        # Сортируем по количеству вхождений
        tag_counts = tag_counts.sort_values(by='count', ascending=False)
        
        # Сохраняем результат в словаре
        tag_counts_results[tag_name] = tag_counts

    return tag_counts_results


<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Обзор колонок датасета

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

<div class="alert alert-info">

##### **Overview (Обзор)**
- <span style="color:#af0400">**title**</span> -  <span style="color:#02020A">**Название стратегии**</span>;
- <span style="color:#af0400">**strategy_description**</span> - <span style="color:#02020A">**Описание стратегии**</span>;
- <span style="color:#af0400">**tags**</span> - <span style="color:#02020A">**Теги к стратегии**</span>;

##### **Performance (Отчет о стратегии)**
- <span style="color:#af0400">**Open P&L**</span> - <span style="color:#02020A">**Анализ сделок**</span> (Прибыль или убыток по текущей открытой позиции. Если позиция не открыта, возвращается значение N/A)</span>;
- <span style="color:#af0400">**Net profit**</span> - <span style="color:#02020A">**Чистая прибыль**</span> (Значение представляет собой сумму всех значений из столбца «Прибыль»);
- <span style="color:#af0400">**Gross profit**</span> - <span style="color:#02020A">**Валовая прибыль**</span> (Общая прибыль по всем прибыльным сделкам, генерируемая стратегией);
- <span style="color:#af0400">**Gross loss**</span> - <span style="color:#02020A">**Валовый убыток**</span> (Общие убытки по всем убыточным сделкам, совершенным по стратегии. Анализ и сокращение торговых убытков — чрезвычайно важная часть анализа торговой стратегии. Именно поэтому эта характеристика стратегии является наиболее важной. Следует отметить, что чистая прибыль увеличивается не только при увеличении валовой прибыли, но и при сокращении валовых убытков);
- <span style="color:#af0400">**Commission paid**</span> - <span style="color:#02020A">**Уплаченная комиссия**</span>;
- <span style="color:#af0400">**Buy & hold returntact**</span> - <span style="color:#02020A">**Возврат средств при покупке и удержании**</span> (Доходность, полученная в случае, если все средства (начальный капитал) были использованы для покупки ценной бумаги при открытии первой сделки и позиция удерживалась в течение всего тестового периода);
- <span style="color:#af0400">**Max equity run-up**</span> - <span style="color:#02020A">**Максимальный прирост собственного капитала**</span> (Отображает максимальный выигрыш, то есть максимально возможный выигрыш, который стратегия могла бы получить по всем совершённым сделкам);
- <span style="color:#af0400">**Max equity drawdown**</span> - <span style="color:#02020A">**Максимальная просадка собственного капитала**</span> (Отображает наибольшую просадку по убыткам, то есть максимально возможный убыток, который могла бы понести стратегия по всем совершённым сделкам.);
- <span style="color:#af0400">**Max contracts held**</span> - <span style="color:#02020A">**Максимальное количество заключенных контрактов**</span> (Максимальное количество контрактов, заключенных одновременно.).


##### **Trades analysis (Анализ сделок)**
- <span style="color:#af0400">**Total trades**</span> - <span style="color:#02020A">**Общее количество сделок**</span> (как выигрышных, так и убыточных);
- <span style="color:#af0400">**Total open trades**</span> - <span style="color:#02020A">**Общее количество открытых сделок**</span> (в данный момент);
- <span style="color:#af0400">**Winning trades**</span> - <span style="color:#02020A">**Выигрышные сделки**</span>;
- <span style="color:#af0400">**Losing trades**</span> - <span style="color:#02020A">**Убыточные сделки**</span>;
- <span style="color:#af0400">**Percent profitable**</span> - <span style="color:#02020A">**Процент прибыльности**</span> (Процент прибыльных сделок, совершённых по стратегии. Рассчитывается путём деления количества прибыльных сделок на общее количество закрытых сделок, совершённых по стратегии. Процент прибыльных сделок сам по себе не является надёжным показателем);
- <span style="color:#af0400">**Avg P&L**</span> - <span style="color:#02020A">**Средний доход за год**</span> (Сумма денег, полученная или потерянная в результате средней сделки, заключённой по стратегии. Рассчитывается путём деления чистой прибыли на общее количество закрытых сделок. Важное значение, поскольку оно должно быть достаточно большим, чтобы покрыть комиссию и издержки, связанные с проскальзыванием, и при этом приносить прибыль);
- <span style="color:#af0400">**Avg winning trade**</span> - <span style="color:#02020A">**Средняя выигрышная сделка**</span> (Валовая прибыль, делённая на количество прибыльных сделок, совершённых по стратегии);
- <span style="color:#af0400">**Avg losing trade**</span> - <span style="color:#02020A">**Средняя убыточная сделка**</span> (Общий убыток, делённый на количество убыточных сделок, открытых по стратегии);
- <span style="color:#af0400">**Ratio avg win / avg loss**</span> - <span style="color:#02020A">**Соотношение avg выигрыш / avg проигрыш**</span> (Среднее значение того, сколько денежных единиц вы выигрываете на каждую потерянную единицу (в выбранной валюте). Это значение рассчитывается путём деления среднего количества выигрышных сделок на среднее количество проигрышных сделок. Само по себе это поле не является очень значимым, поскольку оно не учитывает соотношение количества выигрышных и проигрышных сделок, а стратегии могут иметь разные подходы к прибыльности);
- <span style="color:#af0400">**Largest winning trade**</span> - <span style="color:#02020A">**Самая крупная выигрышная сделка**</span> (Наибольшая денежная прибыль и наибольшая процентная прибыль. Эти показатели обычно относятся к одной и той же сделке, но могут относиться и к разным сделкам);
- <span style="color:#af0400">**Largest winning trade percent**</span> - <span style="color:#02020A">**Процент самой крупной выигрышной сделки**</span>;
- <span style="color:#af0400">**Largest losing trade**</span> - <span style="color:#02020A">**Самая убыточная сделка**</span>;
- <span style="color:#af0400">**Largest losing trade percent**</span> - <span style="color:#02020A">**Процент самой убыточной сделки**</span>(Наибольшая денежная потеря и наибольшая процентная потеря. Эти показатели обычно относятся к одной и той же сделке, но могут относиться и к разным сделкам);
- <span style="color:#af0400">**Avg # bars in trades**</span> - <span style="color:#02020A">**Среднее количество в баров сделок**</span>;
- <span style="color:#af0400">**Avg # bars in winning trades**</span> - <span style="color:#02020A">**Среднее количество в баров в выигрышных сделках**</span>;
- <span style="color:#af0400">**Avg # bars in losing trades**</span> - <span style="color:#02020A">**Среднее количество в баров убыточных сделках**</span>;
 
##### **Risk / performance ratios (Соотношения риска и доходности)**
- <span style="color:#af0400">**Sharpe ratio**</span> - <span style="color:#02020A">**Коэффициент Шарпа**</span> (Чем выше коэффициент Шарпа, тем более плавная кривая доходности. Для многих трейдеров плавная кривая доходности является важной целью);
- <span style="color:#af0400">**Sortino ratio**</span> - <span style="color:#02020A">**Коэффициент Сортино**</span> (Даёт более полное представление об эффективности портфеля с учётом риска, поскольку положительная волатильность считается преимуществом);
- <span style="color:#af0400">**Profit factor**</span> - <span style="color:#02020A">**Коэффициент прибыли**</span> (Сумма денег, которую торговая стратегия заработала на каждую единицу потерянных денег (в выбранной валюте). Это значение рассчитывается путём деления валовой прибыли на валовые убытки);
- <span style="color:#af0400">**Margin calls**</span> - <span style="color:#02020A">**Маржин-коллы**</span> (Общее количество маржин-коллов, сгенерированных стратегией);

##### **Properties (Свойства стратегий)**
- <span style="color:#af0400">**properties**</span> - <span style="color:#02020A">**Свойства**</span> (Содержит в себе диапазон торговых дат, Информации о символах, исходные данные стратегии и свойства стратегии);


<center><p style = "background-color : white; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Дополнительные материалы:</span>

<span style="color:#00FF00">**Коэффициент Шарпа**</span> — это метрика, которая оценивает эффективность инвестиций, учитывая их доходность с поправкой на риск. Он показывает, насколько хорошо актив или стратегия вознаграждает инвестора за принятый риск. Чем выше значение, тем лучше соотношение доходности и риска.
$$
\text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p}
$$
Где:  
- $R_p$ — средняя доходность портфеля/стратегии.  
- $R_f$ — безрисковая ставка (например, доходность гособлигаций).  
- $\sigma_p$ — стандартное отклонение доходности портфеля (волатильность).

**Как интерпретировать коэффициент Шарпа?**

> $>2$ — Отлично: высокая эффективность.

> $>1$ — Хорошо: доходность компенсирует риск.

> $<1$ — Низкая эффективность: риск не оправдан доходностью. 



<span style="color:#00FF00">**Коэффициент Сортино**</span> - это метрика, оценивающая доходность инвестиций с учётом только негативного риска (downside risk). В отличие от коэффициента Шарпа, который учитывает общую волатильность (как рост, так и падение), Сортино фокусируется на риске убытков. Это делает его более подходящим для оценки стратегий, где важно контролировать просадки.
$$
\text{Sortino Ratio} = \frac{R_p - R_f}{\sigma_d}
$$

Где:  
- $R_p$ — средняя доходность портфеля.  
- $R_f$ — безрисковая ставка (например, доходность гособлигаций).  
- $\sigma_d$ — downside deviation (стандартное отклонение только тех доходностей, которые ниже целевого уровня, например, $R_f$).

**Как интерпретировать коэффициент Шарпа?**
> $>2$ — Отлично: Стратегия/актив приносит высокую доходность с минимальными просадками.

> $>1$ — Хорошо: Риск убытков оправдан доходностью.

> $<1$ — Слабый результат: Риск убытков слишком высок относительно доходности. 

> $<0$ - Опасный сигнал: Средняя доходность ниже безрисковой ставки. Инвестор теряет деньги с учётом риска

<span style="color:#00FF00">**Коэффициент прибыли**</span> - показывает соотношение общей прибыли к общим убыткам. Это ключевой показатель для оценки эффективности торговой стратегии.


$$
\text{Profit Factor} = \frac{\sum \text{Profit}}{\sum \text{Loss}}
$$

> $>2$ — Cтратегия генерирует вдвое больше прибыли, чем убытков

> $>1$ — Стратегия прибыльна.

> $=1$ — Безубыточность.

> $<1$ —  Убыточность.


<span style="color:#FF4500">**Сравнение Profit Factor, Коэффициента Шарпа и Коэффициента Сортино**</span>

| Параметр                | Profit Factor                  | Коэффициент Шарпа             | Коэффициент Сортино            |
|-------------------------|--------------------------------|--------------------------------|---------------------------------|
| **Что измеряет**         | Соотношение прибыли и убытков  | Доходность на единицу общего риска | Доходность на единицу downside-риска |
| **Формула**             | (Сумма прибыли) / (Сумма убытков) | (Rp - Rf) / σp                | (Rp - Rf) / σd                  |
| **Учитываемый риск**    | Не учитывает                   | Общая волатильность            | Только негативная волатильность |
| **Идеальное значение**  | ≥ 2.0                         | ≥ 1.5                          | ≥ 2.0                          |
| **Где применять**       | Оценка рентабельности стратегии | Сравнение с рыночными индексами | Контроль просадок              |
| **Пример расчета**      | Прибыль: $15k, Убытки: $5k → PF = 3.0 | Rp=12%, Rf=2%, σp=10% → (12-2)/10 = 1.0 | Rp=15%, Rf=2%, σd=5% → (15-2)/5 = 2.6 |
| **Плюсы**               | Простота интерпретации         | Универсальность                | Фокус на риск убытков           |
| **Минусы**              | Игнорирует размер и частоту сделок | Наказывает за позитивную волатильность | Зависит от выбора MAR*          |
| **Подходит для**        | Начинающих трейдеров           | Долгосрочных инвесторов        | Консервативных стратегий        |

*MAR (Minimum Acceptable Return) — минимально допустимая доходность.

---

### Краткие пояснения:
1. **Profit Factor**  
   - Быстрый "чек-ап" стратегии: показывает, во сколько раз прибыль превышает убытки.  
   - Пример: PF = 2 → на каждые $1 убытка приходится $2 прибыли.

2. **Коэффициент Шарпа**  
   - Универсальный стандарт для сравнения активов.  
   - Пример: Sharpe = 1.5 → доходность на 50% выше общего риска.

3. **Коэффициент Сортино**  
   - Идеален для стратегий с жестким стоп-лоссом.  
   - Пример: Sortino = 3 → доходность в 3 раза превышает риск просадок.



<span style="color:#00FF00">**Маржин-коллы**</span> - это требование брокера к трейдеру пополнить счет или закрыть часть позиций, когда уровень маржи (свободных средств) падает ниже допустимого минимума. Это происходит, если убытки по открытым позициям «съедают» залоговые средства. Игнорирование маржин-колла может привести к принудительному закрытию позиций брокером.

<div class="alert alert-info">
    
Читаем данный после парсинга

In [11]:
df = pd.read_csv('tradingview_strategies_full.csv')

In [12]:
df

,title,strategy_description,tags,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,...,Largest losing trade,Largest losing trade percent,Avg # bars in trades,Avg # bars in winning trades,Avg # bars in losing trades,Sharpe ratio,Sortino ratio,Profit factor,Margin calls,properties
0,Daily Breakout + Daily Shadow By Rouro,This script is a Pine v5 strategy designed to ...,"{'statistics', 'Candlestick analysis', 'Chart ...",0,"+8,375.00","9,973.00","1,598.00",0,"+117,304.17","8,375.00",...,"1,598.00",0.51%,50,63,1,0.203,0.453,6.241,0.0,"{'Trading range': 'Mar 10, 2025, 23:05 — May 0..."
1,Order Block Strategy,Strategy OverviewKey FeaturesOrder Block Detec...,"{'Pivot points and levels', 'Pine utilities', ...","−1,453.09","+9,783.49","9,783.49",0.00001,0,−72.38,"10,667.89",...,0.000004,3.49%,"1,152","1,348",71,0.616,9.392,"1,426,164,099.778",0.0,"{'Trading range': 'Oct 01, 2024, 15:30 — May 0..."
2,EXODUS,EXODUS by (D...,"{'Volume', 'Volatility', 'Oscillators'}",0,"+14,293.23","68,836.40","54,543.18","1,672.77","+5,593,815.28","20,726.51",...,"1,767.68",4.71%,4,5,3,−1.136,−0.782,1.262,0.0,"{'Trading range': 'Jan 30, 2020, 11:00 — May 0..."
3,External Signals Strategy Tester v5,External Signals Strategy Tester v5 – User Gui...,"{'educational', 'Bollinger Bands (BB)', 'Bands...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Trading range': ' — ', 'Backtesting range': ..."
4,Guaranteed Entry Strategy (Every 5 Bars),"//version=5strategy(""Guaranteed Entry Strategy...",{'regressions'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,rt maax EMA cross strategy,this just sample of our strategies we publishe...,"{'BTC', 'sma', 'trend', 'Trend Analysis', 'For...","+39,115.55","+1,066,063.89","1,108,975.30","42,911.41","12,611.77","+9,960,471.92","1,106,625.50",...,"11,085.32",10.95%,766,952,282,0.024,2.104,25.843,0.0,"{'Trading range': 'Feb 01, 1933 — Oct 01, 2024..."
955,SPX Fair Value Strategy Ultimate,This is a strategy using the SPX Fair Value de...,{'Oscillators'},0,"+469,888.27","469,888.27",0,0,"−167,001.45","469,888.27",...,NaN,NaN,NaN,NaN,NaN,0.607,28.369,NaN,NaN,"{'Trading range': 'Jan 03, 2022 — Nov 22, 2022..."
956,Multi Trend Cross Strategy Template,Today I am sharing with the community trend cr...,"{'regressions', 'strategy', 'Trend Analysis', ...",0,"+28,127,665.94","31,327,842.85","3,200,176.91","125,781.93","+13,648,918.30","28,133,021.71",...,"1,242,107.46",15.16%,151,262,40,0.137,1.885,9.789,0.0,"{'Trading range': 'Nov 03, 2010 — Oct 26, 2022..."
957,BOLLY Bands,This is a strategy using Bollinger Bands. The ...,"{'bollingerbandstrategy', 'Bollinger Bands (BB...",0,"+7,998.34","11,559.50","3,561.16","1,448.00","+3,159.02","8,014.34",...,224.51,0.22%,57,51,72,1.367,443.519,3.246,0.0,"{'Trading range': 'Oct 17, 2022, 00:35 — Oct 2..."


<div class="alert alert-info">
    
Разберемся, что с пропусками

<div class="alert alert-danger">
    
Пропуска есть, но обусловимся тем, что у каждой стратегии может быть индивидуальные показатели, и внесение средних, медианных, категориальных данных может ухудшить результаты на обучении. Поэтому мы просто почистим строки где есть пропуска. В случае, если не итоги будут не корректными вернемся на этап пропусков и попробуем их заполнить.

In [13]:
# Почистим от пропусков
df.dropna(inplace=True)

--------------------------------

In [14]:
# Сохраним почищенный датафрейм, потому что наверняка его разрушим
data = df.copy() 

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Приведем текстовые колонки в нормальный вид:</span>

In [15]:
# Применим функцию
# Применяем функцию к каждой колонке отдельно
df['title'] = df['title'].apply(clean_text)
df['strategy_description'] = df['strategy_description'].apply(clean_text)
df['tags'] = df['tags'].apply(clean_text)  # Если 'tags' - это строка, иначе нужно будет обработать по-другому


<div class='alert alert-info'>
Разберемся с тем, как нам разделять стратегии

##### **По временным интервалам** 

- <span style="color:#00FA9A">**Краткосрочные (скальпинг, внутридневная торговля)**</span>
- <span style="color:#00FA9A">**Среднесрочные (свинг-трейдинг)**</span>
- <span style="color:#00FA9A">**Долгосрочные (позиционная торговля)**</span>

##### **По методам анализа**
- <span style="color:#00FA9A">**Стратегии на основе технического анализа (индикаторы, паттерны)**</span>
- <span style="color:#00FA9A">**Стратегии на основе объема**</span>
- <span style="color:#00FA9A">**Стратегии на основе ценового действия (Price Action)**</span>
- <span style="color:#00FA9A">**Комбинированные стратегии**</span>

##### **По применяемым индикаторам**
- <span style="color:#00FA9A">**Трендовые (MA, MACD, ADX)**</span>
- <span style="color:#00FA9A">**Осцилляторы (RSI, Stochastic)**</span>
- <span style="color:#00FA9A">**Волатильность (Bollinger Bands, ATR)**</span>
- <span style="color:#00FA9A">**Объемные (OBV, Money Flow Index)**</span>

##### **По механизму входа/выхода**
- <span style="color:#00FA9A">**Пробойные стратегии**</span>
- <span style="color:#00FA9A">**Стратегии возврата к среднему**</span>
- <span style="color:#00FA9A">**Стратегии следования за трендом**</span>
- <span style="color:#00FA9A">**Контртрендовые стратегии**</span>
##### **По типу рынк**
- <span style="color:#00FA9A">**Для акций**</span>
- <span style="color:#00FA9A">**Для криптовалют**</span>
- <span style="color:#00FA9A">**Для фьючерсов**</span>
- <span style="color:#00FA9A">**Для форекс**</span>

<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Описание стратегий

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

<div class='alert alert-info'>

Внутри каждой стратегии находятся дополнительный подтипы, разделения на внутренние подтипы не увенчались успехом и получилось много различных и сложных комбинаций разделений стратегий. В связи с этим мы оставили только основные типы, без внутренних разделений и **создали словарь для классификации стратегий со всеми возможными тегами и словам**.

*Ранее этот словарь был наполнен большим количеством тегов, но после анализа и исключений неопределенных слов приобрел данный вид*

In [16]:
# Новый словарь тегов
tags = {
    "time": ['position', 'momentum', 'shortterm', 'trendfollowing', 'swing', 'longterm',
             'intraday', 'mediumterm', 'scalp', 'invest', 'highfrequency', 'daytrade', 
             'multiday', 'trendfollow'],
    'analysis_Method': ['indicator', 'atr', 'technical', 'volume', 'candlestick', 'liquidity',
                        'fundamental', 'fibonacci', 'vwap', 'supportresistance', 'engulfing', 
                        'earnings', 'obv', 'priceaction', 'harmonic', 'sr', 'cvd', 'pe', 
                        'valuation', 'financials'],
    'indicators': ['ema', 'atr', 'rsi', 'sma', 'ma', 'macd', 'bollinger', 'oscillator',
                   'supertrend', 'adx', 'stochastic', 'breadth', 'fibonacci', 'cci', 
                   'keltner', 'ichimoku', 'distribution', 'roc', 'decline', 'obv', 
                   'sar', 'advance', 'accumulation', 'zlsma', 'mfi', 'dpo', 
                   'movingaverage', 'chaikin'],
    'mechanism': ['reversal', 'oversold', 'overbought', 'breakout', 'divergence', 
                       'reversion', 'countertrend', 'movingaveragecrossover', 'channelbreak', 
                       'trendfollow'],
    'market': ['future', 'equity', 'crypto', 'forex', 'bitcoin', 'currency', 
                    'fx', 'contango', 'backwardation', 'liquidation', 'hodl', 'altcoin']
}

In [17]:
# Выполним классификацию стратегий на основе тегов
df['classification'] = df.apply(lambda x: classify_tags(x, tags), axis=1)

In [18]:
# Узнаем сколько в каждом классе находится стартегий
cl_df = df['classification'].value_counts()
# Строим график
pie_graph(cl_df, 'Соотношение типов стратегий')

<div class="alert alert-success">
    
**Как мы видим из графика чаще всего типы стартегий комбинируются, и сложно выделить стратегию в определенный класс.**

*Данное утверждение опирается исключительно из информации по описанию стратегии.Ну и в целом, логично комбинировать разные типы, чтобы достичь лучшего результата.*

<div class='alert alert-info'>
Теперь разберем вхождение терминов по отдельным направлениям стратегий:

In [19]:
# Вызовем раннюю написанную функцию, для разделения
tag_counts_results = process_tags(df, tags)

# Создадим новые датафреймы для удобства по каждому классу
df_time = tag_counts_results['time']
df_method = tag_counts_results['analysis_Method']
df_indicator = tag_counts_results['indicators']
df_mechanism = tag_counts_results['mechanism']
df_market = tag_counts_results['market']

In [20]:
# Список новых датафреймов
list_df = [df_time, df_indicator, df_market, df_mechanism, df_method]
# Название графиков
title_name = ['time_tags', 'indicator_tags', 'market_tags', 'mechanism_tags', 'method_tags']
# Строим графики по новым датафреймам:
for list, name in zip(list_df, title_name):
    build_graph('px', 'bar', list, 'tag', 'count', 20, 8, name, 'Теги', 'Количество', color='tag')

<div class="alert alert-success">
    
Из созданных датафреймов получили визуализацию вхождения каждых тегов, какие теги чаще входят в ту или иную стратегию, это нам в будущем поможет определиться с лучшей стратегией. Мы определим, какие стратегии наиболее выгодные с лучшей прибылью, и к чему какая применяется, проанализируем эти эффективные стратегии на вхождение тегов, в большой части на вхождение индикаторов, определим, какие индикаторы и методы чаще используются в этих эффективных стратегиях и сможем дать рекомендации по выбору стратегии и её построению.

In [21]:
df

,title,strategy_description,tags,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,...,Avg # bars in winning trades,Avg # bars in losing trades,Sharpe ratio,Sortino ratio,Profit factor,Margin calls,properties,classification,combined,type_tags
0,daily breakout daily shadow by rouro,this script is a pine v5 strategy designed to ...,statistics candlestick analysis chart patterns,0,"+8,375.00","9,973.00","1,598.00",0,"+117,304.17","8,375.00",...,63,1,0.203,0.453,6.241,0.0,"{'Trading range': 'Mar 10, 2025, 23:05 — May 0...",combined strategy,this script is a pine v5 strategy designed to ...,[]
1,order block strategy,strategy overviewkey featuresorder block detec...,pivot points and levels pine utilities volume,"−1,453.09","+9,783.49","9,783.49",0.00001,0,−72.38,"10,667.89",...,"1,348",71,0.616,9.392,"1,426,164,099.778",0.0,"{'Trading range': 'Oct 01, 2024, 15:30 — May 0...",combined strategy,strategy overviewkey featuresorder block detec...,[]
2,exodus,exodus by dafe trading systemsexodus is a soph...,volume volatility oscillators,0,"+14,293.23","68,836.40","54,543.18","1,672.77","+5,593,815.28","20,726.51",...,5,3,−1.136,−0.782,1.262,0.0,"{'Trading range': 'Jan 30, 2020, 11:00 — May 0...",combined strategy,exodus by dafe trading systemsexodus is a soph...,[future]
5,arrows flexible ma cross strategy api ready,arrows highfrequency ma cross scalper api read...,bitcoin cryptocurrency exponential moving aver...,+2.24,+184.19,360.35,176.15,46.20,+103.02,212.07,...,109,43,0.368,0.994,2.046,0.0,"{'Trading range': 'Mar 24, 2025, 16:50 — May 0...",combined strategy,arrows highfrequency ma cross scalper api read...,"[future, crypto, bitcoin, currency]"
6,1h liquidity swings strategy with 12 rr,luxalgo liquidity swings simulateduses tapivot...,breadth indicators,"−1,160.00","+117,390.00","859,610.00","742,220.00",0,"+708,340.00","155,735.00",...,23,17,0.146,0.27,1.158,0.0,"{'Trading range': 'Jan 04, 2023, 14:00 — May 0...",combined strategy,luxalgo liquidity swings simulateduses tapivot...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,3commas bot dca backtester signals free,this is a dca strategy backtester signals buil...,3commasbacktester dcabot 3commasbotsignals dca...,−0.44,+288.93,647.28,358.35,66.71,"−1,880.26",530.92,...,152,"1,078",0.521,21.632,1.806,0.0,"{'Trading range': 'Oct 10, 2022, 00:40 — Oct 2...",combined strategy,this is a dca strategy backtester signals buil...,"[future, crypto, forex]"
954,rt maax ema cross strategy,this just sample of our strategies we publishe...,btc sma trend trend analysis forex usd exponen...,"+39,115.55","+1,066,063.89","1,108,975.30","42,911.41","12,611.77","+9,960,471.92","1,106,625.50",...,952,282,0.024,2.104,25.843,0.0,"{'Trading range': 'Feb 01, 1933 — Oct 01, 2024...",combined strategy,this just sample of our strategies we publishe...,"[future, crypto, forex]"
956,multi trend cross strategy template,today i am sharing with the community trend cr...,regressions strategy trend analysis moving ave...,0,"+28,127,665.94","31,327,842.85","3,200,176.91","125,781.93","+13,648,918.30","28,133,021.71",...,262,40,0.137,1.885,9.789,0.0,"{'Trading range': 'Nov 03, 2010 — Oct 26, 2022...",combined strategy,today i am sharing with the community trend cr...,[]
957,bolly bands,this is a strategy using bollinger bands the s...,bollingerbandstrategy bollinger bands bb bands...,0,"+7,998.34","11,559.50","3,561.16","1,448.00","+3,159.02","8,014.34",...,51,72,1.367,443.519,3.246,0.0,"{'Trading range': 'Oct 17, 2022, 00:35 — Oct 2...",indicators,this is a strategy using bollinger bands the s...,[]


<center><p style = "background-color : white; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Risk / performance ratios (Соотношения риска и доходности)</span>

In [22]:
# df_ratio = data[['title','Sharpe ratio', 'Sortino ratio', 'Profit factor', 'Margin calls']]

In [23]:
# df_ratio

Надо числовые колонки привести подобающий вид.
Колонки `['title', 'Margin calls']` - останутся как есть

In [24]:
df[['Sharpe ratio', 'Sortino ratio', 'Profit factor']] = df[['Sharpe ratio', 'Sortino ratio', 'Profit factor']].map(convert_to_numeric)

In [25]:
# df_ratio.describe()

<div class="alert alert-danger">
Анализируя статистические показатели Коэффициента Шарпа, Сортино и профит фактора, можно сказать что в данных присутвуют Аномально высокие значения стандартного отклонения и отрицательные минимумы указывают на наличие серьезных проблем с данными или расчетами. Надо либо исключать эти стратегии из анализа, либо проверить корректность работы и расчёта формул на сайте по этим коэффициентам.


**В связи с этим, выполним работу по чистке стратегий с аномальными значениями, не смотря на то, что для этих коэффициентов нормальные диапазоны находятся в пределах от -1 до 2. Мы сделаем скидку и оставим стратегии с пределами коэффициентов от -3 до 3**

In [26]:
df = df[
    (df['Sharpe ratio'].between(-3, 3)) &
    (df['Sortino ratio'].between(-3, 3)) &
    (df['Profit factor'].between(-3, 3))
]

# print(f'Количество аномальных стратегий: {df.shape[0]-df.shape[0]}')

In [27]:
# df_r = df_ratio[
#     (df_ratio['Sharpe ratio'].between(-3, 3)) &
#     (df_ratio['Sortino ratio'].between(-3, 3)) &
#     (df_ratio['Profit factor'].between(-3, 3))
# ]

# print(f'Количество аномальных стратегий: {df_ratio.shape[0]-df_r.shape[0]}')

In [28]:
df['Margin calls'].value_counts()

Margin calls
0.0      550
1.0        3
320.0      1
290.0      1
567.0      1
28.0       1
209.0      1
7.0        1
38.0       1
Name: count, dtype: int64

<div class="alert alert-danger">


**В целом у Margin calls не особо много разных значений, в основном все нулевые, и я бы дропнул эту колонку и убрал с изучения**

<div class="alert alert-info">
    
Разберемся теперь с остальными коэффициентами и глянем, что из себя представляют

In [29]:
df_r = df[['title','Sharpe ratio', 'Sortino ratio', 'Profit factor']]

In [30]:
df_r

,title,Sharpe ratio,Sortino ratio,Profit factor
2,exodus,-1.136,-0.782,1.262
5,arrows flexible ma cross strategy api ready,0.368,0.994,2.046
6,1h liquidity swings strategy with 12 rr,0.146,0.270,1.158
8,3070 rsi strategy with colored bar,0.007,0.010,1.124
10,parabolic rsi strategy ma filter tpsl pakunfx,-0.180,-0.236,1.426
...,...,...,...,...
948,simple rsi and sma long and short by coinrule,0.234,0.409,1.356
949,50 pips a day strategy kaspricci,-0.038,-0.055,1.058
950,strategy weekly results as numbers v1,0.197,0.499,1.140
951,heikin ashi supertrend,0.312,2.367,2.255


In [31]:
grap_list = ['Sharpe ratio', 'Sortino ratio', 'Profit factor']
for grap in grap_list:
    gist_graph(df_r, grap, grap)

<div class="alert alert-info">
    
Выделим топ 20 стратегий по каждому из показателей

In [32]:
sharpe = df_r.sort_values(by = 'Sharpe ratio', ascending=False).head(20)
sortino = df_r.sort_values(by = 'Sortino ratio', ascending=False).head(20)
profit = df_r.sort_values(by = 'Profit factor', ascending=False).head(20)


In [33]:
# Список новых датафреймов
list_df = [sharpe, sortino, profit]
# Название графиков
title_name = ['Sharpe ratio', 'Sortino ratio', 'Profit factor']
# Строим графики по новым датафреймам:
for list, name in zip(list_df, title_name):
    build_graph('px', 'bar', list, 'title', name, 20, 15, name, '', 'Коэффициент', color='title')

<div class="alert alert-info">
На графиках видно, что среди трех коэффициентов, предназначенных для оценки доходности инвестиций, есть пересечения между стратегиями. Но если рассматривать их в категории двух разных коэффициентов, в случае рассмотрения трех сразу у нас пересечений нет.
</div>

<div class="alert alert-danger">

**Возможно, дело в том, что мы делали скидку и оставили стратегии с пределами коэффициентов от -3 до 3. Возможно, стоит поработать с этим коэффициентом, и тогда найдется пересечение, или увеличить количество топовых стратегий по показателям с 20 до 100.**
</div>

<div class="alert alert-info">
Так как Коэффициент Шарпа и Коэффициент Сортино учитывают риски волатильности и оба измеряют доходность, то выделим на их основе самые топовые стратегии:
</div>


In [34]:
top_20_strategy = pd.merge(sharpe[['title']], sortino[['title']], on='title', how='inner')
top_20_strategy

,title
0,psar bbpt zlsma btc 1min
1,overnight effect high volatility crypto aibitc...
2,dual strategy selector v2 cryptogyani
3,bollinger bands breakout strategy
4,trendguard scalper ssl hama candle with consol...
5,strategy sema sdi webhook
6,ta strategy
7,postopen long strategy with atrbased stop loss...


<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Выбор стратегий исходя из показателей прибыльности

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

<div class="alert alert-info">
    
Из всех показателей данных блоков для оценки прибыльности торговой стратегии на TradingView наиболее важными показателями являются:

- Net profit – Чистая прибыль, основной показатель успешности стратегии.

- Sharpe ratio – Чем выше коэффициент Шарпа, тем более плавная кривая доходности. Для многих трейдеров плавная кривая доходности является важной целью.

- Percent profitable – Процент прибыльных сделок, важный параметр устойчивости стратегии.

- Ratio avg win / avg loss – Соотношение средней прибыли к среднему убытку (важно для оценки риск-менеджмента).

- Max equity drawdown – Максимальная просадка, критичен для оценки рисков.

Остальные показатели (например, Max contracts held, Total open trades) больше относятся к управлению позициями, а не к прямой оценке прибыльности.

In [35]:
numeric_cols = ['Net profit', 'Sharpe ratio', 'Percent profitable', 'Ratio avg win / avg loss', 'Max equity drawdown', 'Total trades']

df[numeric_cols] = df[numeric_cols].applymap(convert_to_numeric)
df.head()

C:\Users\xndrf\AppData\Local\Temp\ipykernel_35728\1771303511.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\xndrf\AppData\Local\Temp\ipykernel_35728\1771303511.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,title,strategy_description,tags,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,...,Avg # bars in winning trades,Avg # bars in losing trades,Sharpe ratio,Sortino ratio,Profit factor,Margin calls,properties,classification,combined,type_tags
2,exodus,exodus by dafe trading systemsexodus is a soph...,volume volatility oscillators,0,14293.23,"68,836.40","54,543.18","1,672.77","+5,593,815.28","20,726.51",...,5,3,-1.136,-0.782,1.262,0.0,"{'Trading range': 'Jan 30, 2020, 11:00 — May 0...",combined strategy,exodus by dafe trading systemsexodus is a soph...,[future]
5,arrows flexible ma cross strategy api ready,arrows highfrequency ma cross scalper api read...,bitcoin cryptocurrency exponential moving aver...,+2.24,184.19,360.35,176.15,46.20,+103.02,212.07,...,109,43,0.368,0.994,2.046,0.0,"{'Trading range': 'Mar 24, 2025, 16:50 — May 0...",combined strategy,arrows highfrequency ma cross scalper api read...,"[future, crypto, bitcoin, currency]"
6,1h liquidity swings strategy with 12 rr,luxalgo liquidity swings simulateduses tapivot...,breadth indicators,"−1,160.00",117390.00,"859,610.00","742,220.00",0,"+708,340.00","155,735.00",...,23,17,0.146,0.270,1.158,0.0,"{'Trading range': 'Jan 04, 2023, 14:00 — May 0...",combined strategy,luxalgo liquidity swings simulateduses tapivot...,[]
8,3070 rsi strategy with colored bar,this script colors price bars based on relativ...,forecasting pine utilities moving averages,0,4755.16,"42,956.67","38,201.51",0,"−126,495.49","16,724.65",...,108,251,0.007,0.010,1.124,0.0,"{'Trading range': 'Feb 18, 2025, 23:42 — May 0...",combined strategy,this script colors price bars based on relativ...,[]
10,parabolic rsi strategy ma filter tpsl pakunfx,parabolic rsi strategy ma filter tpslpakunfxth...,exponential moving average ema strategy strate...,0,29.75,99.62,69.87,4.81,−442.20,39.31,...,101,138,-0.180,-0.236,1.426,0.0,"{'Trading range': 'Sep 30, 2024, 10:50 — May 0...",combined strategy,parabolic rsi strategy ma filter tpslpakunfxth...,"[forex, fx]"


In [36]:
df = df[ df['Total trades'] > 100]

print(df.shape[0])

408


In [37]:
dff = df[numeric_cols]

In [38]:
dff.describe()

,Net profit,Sharpe ratio,Percent profitable,Ratio avg win / avg loss,Max equity drawdown,Total trades
count,4.080000e+02,408.000000,408.000000,408.000000,4.080000e+02,408.000000
mean,4.398695e+09,-0.019926,0.481983,1.786662,4.925728e+08,504.448529
std,8.923864e+10,0.543689,0.158500,1.172057,9.524662e+09,968.784792
min,-8.348381e+09,-2.665000,0.114600,0.031000,2.000000e-02,101.000000
25%,4.119350e+02,-0.134000,0.372475,0.957750,4.121750e+02,150.000000
50%,5.340370e+03,0.136500,0.447400,1.582000,2.875065e+03,252.000000
75%,5.412076e+04,0.300750,0.602125,2.242000,3.309438e+04,528.500000
max,1.802494e+12,0.766000,0.962300,11.413000,1.922195e+11,12301.000000


In [39]:
for grap in numeric_cols:
    gist_graph(dff, grap, grap)

In [40]:
max_net_profit = df['Net profit'].max()

In [41]:
df

,title,strategy_description,tags,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,...,Avg # bars in winning trades,Avg # bars in losing trades,Sharpe ratio,Sortino ratio,Profit factor,Margin calls,properties,classification,combined,type_tags
2,exodus,exodus by dafe trading systemsexodus is a soph...,volume volatility oscillators,0,14293.23,"68,836.40","54,543.18","1,672.77","+5,593,815.28","20,726.51",...,5,3,-1.136,-0.782,1.262,0.0,"{'Trading range': 'Jan 30, 2020, 11:00 — May 0...",combined strategy,exodus by dafe trading systemsexodus is a soph...,[future]
6,1h liquidity swings strategy with 12 rr,luxalgo liquidity swings simulateduses tapivot...,breadth indicators,"−1,160.00",117390.00,"859,610.00","742,220.00",0,"+708,340.00","155,735.00",...,23,17,0.146,0.270,1.158,0.0,"{'Trading range': 'Jan 04, 2023, 14:00 — May 0...",combined strategy,luxalgo liquidity swings simulateduses tapivot...,[]
10,parabolic rsi strategy ma filter tpsl pakunfx,parabolic rsi strategy ma filter tpslpakunfxth...,exponential moving average ema strategy strate...,0,29.75,99.62,69.87,4.81,−442.20,39.31,...,101,138,-0.180,-0.236,1.426,0.0,"{'Trading range': 'Sep 30, 2024, 10:50 — May 0...",combined strategy,parabolic rsi strategy ma filter tpslpakunfxth...,"[forex, fx]"
13,ut bot strategy backtest with date range,backtesting strategy for ut bot alerts by quan...,portfolio management educational strategy stat...,−0.04,-31.28,364.15,395.43,0,−4.21,55.25,...,16,6,-0.099,-0.140,0.921,0.0,"{'Trading range': 'Apr 01, 2025, 19:05 — May 0...",combined strategy,backtesting strategy for ut bot alerts by quan...,[equity]
17,bb 202 rsi10 stoch 1433 5min,on the 1minute chart using the indicators stoc...,options pine utilities bands and channels,−183.40,8562.03,"29,730.87","21,168.84",0,"+144,746.73","11,117.67",...,86,198,0.524,1.040,1.404,0.0,"{'Trading range': 'Apr 20, 2025, 19:02 — May 0...",indicators,on the 1minute chart using the indicators stoc...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,fft strategy bidirectional stopprofittrailing ...,this strategy uses the fast fourier transform ...,trailingstop centered oscillators fouriertrans...,0,1898846.87,"4,933,839.69","3,034,992.82",0,"+16,457.36","2,148,092.97",...,53,14,0.371,1.697,1.626,0.0,"{'Trading range': 'Dec 05, 2013, 04:00 — Nov 1...",combined strategy,this strategy uses the fast fourier transform ...,[]
949,50 pips a day strategy kaspricci,50 pips a day strategythis strategy is designe...,strategy candlestick analysis,0,28.63,520.36,491.73,0,−156.99,56.52,...,9,4,-0.038,-0.055,1.058,0.0,"{'Trading range': 'Jan 04, 2021, 10:00 — Nov 1...",combined strategy,50 pips a day strategythis strategy is designe...,[forex]
950,strategy weekly results as numbers v1,this script is based on an idea of monthly sta...,weekly charts statistic statistics pnl,"+174,070.60",154092.57,"1,255,034.89","1,100,942.32","29,334.81","+1,174,770.85","503,443.39",...,66,24,0.197,0.499,1.140,0.0,"{'Trading range': 'Jan 01, 2021, 21:00 — Nov 0...",No classification,this script is based on an idea of monthly sta...,[]
951,heikin ashi supertrend,about this strategythis supertrend strategy us...,cycles alertatron heikinashi supertrend automa...,+32.61,1405.45,"2,525.37","1,119.92",31.29,"+1,318.97","1,464.67",...,57,23,0.312,2.367,2.255,0.0,"{'Trading range': 'Jan 27, 2019, 21:00 — Nov 0...",combined strategy,about this strategythis supertrend strategy us...,[equity]


Поскольку показатели имеют разный масштаб (например, Net Profit — в долларах, а Sharpe Ratio — безразмерный), их нужно нормализовать.

In [42]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
# Масштабируем данные
scaler = MinMaxScaler()
dff[numeric_cols] = scaler.fit_transform(df[numeric_cols])  # Используем df для масштабирования

C:\Users\xndrf\AppData\Local\Temp\ipykernel_35728\3775584429.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
# Кластеризация
X = dff[numeric_cols]
kmeans = KMeans(n_clusters=3, random_state=42)  # Разделим на 3 группы
dff['Cluster'] = kmeans.fit_predict(X)

# Добавляем колонку 'Cluster' в основной DataFrame df
df['Cluster'] = dff['Cluster']


C:\Users\xndrf\AppData\Local\Temp\ipykernel_35728\417058588.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
import plotly.express as px

# Создаем интерактивный график с заданными размерами
fig = px.scatter(df, 
                 x='Sharpe ratio', 
                 y='Net profit', 
                 color='Cluster', 
                 title='Кластеризация стратегий',
                 labels={'Cluster': 'Кластер'},
                 hover_data=['Percent profitable', 'Max equity drawdown', 'Total trades'])

# Устанавливаем размеры графика
fig.update_layout(width=800, height=600)  # Задайте нужные размеры в пикселях

# Обновляем внешний вид графика
fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.8,
        line=dict(width=1, color='black')
    )
)
# Или задать свою палитру
fig.update_layout(coloraxis=dict(colorscale='Viridis'))

# Показываем график
fig.show()


In [45]:
df['Cluster'].value_counts()

Cluster
0    213
1    147
2     48
Name: count, dtype: int64

Проведем кластерный анализ для выявления паттернов.

Оптимальное количество кластеров для анализа стратегий — 3. Это позволяет разделить стратегии на:

Высокодоходные с низким риском (лучшие).

Средние по доходности и риску.

Низкодоходные или высокорисковые (худшие).

# Взвешивание показателей

Не все показатели одинаково важны. Например, Sharpe Ratio и Max Drawdown важнее в нашем случае, чем Total Trades.

Сделать только кластеры = 0

In [46]:
# Задаём веса (сумма = 1)
weights = {
    'Net profit': 0.2,
    'Sharpe ratio': 0.3,  # Чем выше, тем лучше
    'Percent profitable': 0.1,
    'Ratio avg win / avg loss': 0.1,
    'Max equity drawdown': 0.2,  # Чем меньше просадка, тем лучше → можно инвертировать
    'Total trades': 0.1   # Чем больше сделок, тем надёжнее статистика
}

# Инвертируем Max Drawdown (меньше просадка → выше оценка)
df['Max equity drawdown'] = 1 - df['Max equity drawdown']

# Рассчитываем общий рейтинг
df['Score'] = (
    df['Net profit'] * weights['Net profit'] +
    df['Sharpe ratio'] * weights['Sharpe ratio'] +
    df['Percent profitable'] * weights['Percent profitable'] +
    df['Ratio avg win / avg loss'] * weights['Ratio avg win / avg loss'] +
    df['Max equity drawdown'] * weights['Max equity drawdown'] +
    df['Total trades'] * weights['Total trades']
)

# Сортируем по убыванию рейтинга
best_strategies = df.sort_values('Score', ascending=False)
display(best_strategies.head(10))

,title,strategy_description,tags,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,...,Sharpe ratio,Sortino ratio,Profit factor,Margin calls,properties,classification,combined,type_tags,Cluster,Score
331,4vietnamese 3x supertrend,this strategy attempts to capture long positio...,supertrend trend analysis,"+14,066,337,919.24",1.802494e+12,"2,841,920,925,552.06","1,039,427,424,569.39","45,286,771,092.96","+95,435,241,337.54","1,901,121,051,441.89",...,0.214,1.960,2.734,0.0,"{'Trading range': 'Sep 18, 2000 — Jan 06, 2025...",combined strategy,this strategy attempts to capture long positio...,[],0,3.220548e+11
258,jma quantum edge adaptive precision trading sy...,jma quantum edge adaptive precision trading sy...,jurik portfolio management multitimeframe,0,8.547079e+07,"163,784,981.09","78,314,195.90","211,585.06","+191,692,000.00","89,420,170.43",...,0.224,1.337,2.091,0.0,"{'Trading range': 'Sep 29, 2011 — Feb 13, 2025...",combined strategy,jma quantum edge adaptive precision trading sy...,[equity],1,8.521120e+06
302,internal bar strength ibs strategy,strategy description the internal bar strength...,cycles reversal qqq stocks buythedip meanrever...,0,4.731337e+07,"96,610,134.93","49,296,767.81",0,"+9,245,702.34","48,237,066.52",...,0.243,0.448,1.960,0.0,"{'Trading range': 'Mar 19, 1999 — Jan 21, 2025...",combined strategy,strategy description the internal bar strength...,[],1,8.146611e+06
315,sunil 2 bar breakout strategy,detailed explanation of the sunil 2 bar breako...,candlestick analysis,0,3.284367e+07,"169,497,543.50","136,653,868.70",0,"+10,365,088.50","35,069,882.25",...,0.304,0.747,1.240,0.0,"{'Trading range': 'Feb 03, 2009, 11:15 — Jan 1...",combined strategy,detailed explanation of the sunil 2 bar breako...,"[equity, crypto, forex, bitcoin]",0,5.601453e+06
311,consecutive bearish candle strategy,strategy description the consecutive bearish c...,mean reversal qqq daytrading trend analysis re...,0,3.016888e+07,"45,395,232.11","15,226,352.28",0,"+9,751,607.13","30,238,250.13",...,0.317,0.663,2.981,0.0,"{'Trading range': 'Mar 23, 1999 — Jan 21, 2025...",combined strategy,strategy description the consecutive bearish c...,[],1,5.406303e+06
301,3 down 3 up strategy,strategy description the 3 down 3 up strategy ...,cycles qqq buythedip meanreversion spdr sampp ...,"+631,347.20",2.762227e+07,"55,344,355.97","27,722,086.24",0,"+15,605,892.90","29,129,678.45",...,0.297,0.522,1.996,0.0,"{'Trading range': 'Jan 13, 2005, 18:30 — Jan 2...",combined strategy,strategy description the 3 down 3 up strategy ...,[],1,5.093823e+06
309,consecutive bars abovebelow ema buy the dip st...,strategy description the consecutive bars abov...,cycles movingavarage qqq reversal meanreversio...,0,2.507903e+07,"41,370,037.50","16,291,008.94",0,"+9,751,607.13","25,137,071.36",...,0.295,0.502,2.539,0.0,"{'Trading range': 'Mar 23, 1999 — Jan 21, 2025...",combined strategy,strategy description the consecutive bars abov...,[],1,4.668084e+06
134,50 ema crossover with monthly dca,recommended chart interval 1woverviewthis stra...,portfolio management exponential moving averag...,0,2.656447e+07,"40,708,165.24","14,143,696.05",0,"+4,515,399.94","29,497,353.35",...,0.086,1.211,2.878,0.0,"{'Trading range': 'Jan 07, 1980 — Mar 31, 2025...",combined strategy,recommended chart interval 1woverviewthis stra...,[],0,4.322264e+06
305,3bar low strategy,strategy description the 3bar low strategy is ...,cycles reversal qqq buythedip meanreversion da...,0,2.311899e+07,"47,534,392.07","24,415,405.06",0,"+12,502,910.49","23,565,565.34",...,0.207,0.296,1.947,0.0,"{'Trading range': 'Feb 09, 1993 — Jan 21, 2025...",combined strategy,strategy description the 3bar low strategy is ...,[],1,3.740881e+06
236,squeeze momentum indicator strategy lazybear p...,the squeeze momentum indicator strategy sqzmom...,pineindicators johnfcarter sqzmom_lb squeezemo...,0,2.297303e+07,"59,966,419.04","36,993,387.63",0,"+1,305,469.33","32,766,669.80",...,0.3